## code (pytorch version)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AWGN(nn.Module):
    r"""AWGN(dtype=torch.complex64, **kwargs)

    Add complex AWGN to the inputs with a certain variance.

    This class inherits from the PyTorch `nn.Module` class and can be used as a layer in
    a PyTorch model.

    This layer adds complex AWGN noise with variance `no` to the input.
    The noise has variance `no/2` per real dimension.
    It can be either a scalar or a tensor which can be broadcast to the shape
    of the input.

    Parameters
    ----------
        dtype : Complex torch.dtype
            Defines the datatype for internal calculations and the output
            dtype. Defaults to `torch.complex64`.

    Input
    -----

        (x, no) :
            Tuple:

        x :  Tensor, torch.complex
            Channel input

        no : Scalar or Tensor, torch.float
            Scalar or tensor whose shape can be broadcast to the shape of `x`.
            The noise power `no` is per complex dimension. If `no` is a
            scalar, noise of the same variance will be added to the input.
            If `no` is a tensor, it must have a shape that can be broadcast to
            the shape of `x`. This allows, e.g., adding noise of different
            variance to each example in a batch. If `no` has a lower rank than
            `x`, then `no` will be broadcast to the shape of `x` by adding
            dummy dimensions after the last axis.

    Output
    -------
        y : Tensor with same shape as `x`, torch.complex
            Channel output
    """

    def __init__(self, dtype=torch.complex64, **kwargs):
        super().__init__()
        self.dtype = dtype

    def forward(self, inputs):
        x, no = inputs

        # Create real-valued Gaussian noise for each complex dimension
        noise_real = torch.randn_like(x.real)
        noise_imag = torch.randn_like(x.imag)
        noise = noise_real + 1j * noise_imag

        # Scale noise according to variance
        scale = torch.sqrt(no / 2)
        noise *= scale.unsqueeze(-1)  # broadcast over last dimension

        # Convert noise to complex tensor
        noise = torch.complex(noise.real, noise.imag)

        # Add noise to input
        y = x + noise.to(self.dtype)

        return y


## example(pytorch version)

In [3]:
# example:
awgn_channel = AWGN()
x = torch.randn(10, 2, 2, 14, 64, dtype=torch.complex64)
no = torch.tensor(0.01, dtype=torch.float32)
y = awgn_channel((x, no))
print(y)


tensor([[[[[ 2.6591e-01-8.3063e-01j, -1.3320e+00+5.6904e-01j,
             1.7742e-01-2.4942e-01j,  ...,
            -2.5938e-01+4.1941e-01j,  5.8936e-01-4.1055e-01j,
             2.4924e-02+6.1776e-01j],
           [-7.6309e-01+7.6502e-01j,  4.4790e-01-7.8986e-01j,
            -5.1406e-01+8.3553e-01j,  ...,
             4.9114e-01+2.0791e-01j, -1.1754e+00-1.6621e+00j,
            -6.6864e-01+7.8388e-01j],
           [-9.1479e-01-1.1988e+00j,  5.3080e-01-7.4044e-01j,
             4.0311e-01-7.9516e-02j,  ...,
             7.0348e-01+9.7761e-01j,  6.5342e-01-2.3063e-01j,
             9.7159e-01-1.7493e-01j],
           ...,
           [ 5.7415e-01-4.8665e-01j,  1.0223e-01+2.1615e-01j,
             2.9220e-01+2.7598e-01j,  ...,
             8.6975e-01-2.2713e-01j,  6.3328e-01-1.3544e-01j,
             2.8187e-01-5.0806e-01j],
           [-6.5401e-01-5.0819e-05j,  5.8776e-01-9.5041e-01j,
            -2.2018e-01-3.6591e-01j,  ...,
             1.3181e+00+5.5226e-01j,  1.0413e-01-6.9330e-01

## code(tf version)

In [4]:
#
# SPDX-FileCopyrightText: Copyright (c) 2021-2023 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
"""Layer for simulating an AWGN channel"""

import tensorflow as tf
from tensorflow.keras.layers import Layer
from sionna.utils import expand_to_rank, complex_normal

class AWGN(Layer):
    r"""AWGN(dtype=tf.complex64, **kwargs)

    Add complex AWGN to the inputs with a certain variance.

    This class inherits from the Keras `Layer` class and can be used as layer in
    a Keras model.

    This layer adds complex AWGN noise with variance ``no`` to the input.
    The noise has variance ``no/2`` per real dimension.
    It can be either a scalar or a tensor which can be broadcast to the shape
    of the input.

    Example
    --------

    Setting-up:

    >>> awgn_channel = AWGN()

    Running:

    >>> # x is the channel input
    >>> # no is the noise variance
    >>> y = awgn_channel((x, no))

    Parameters
    ----------
        dtype : Complex tf.DType
            Defines the datatype for internal calculations and the output
            dtype. Defaults to `tf.complex64`.

    Input
    -----

        (x, no) :
            Tuple:

        x :  Tensor, tf.complex
            Channel input

        no : Scalar or Tensor, tf.float
            Scalar or tensor whose shape can be broadcast to the shape of ``x``.
            The noise power ``no`` is per complex dimension. If ``no`` is a
            scalar, noise of the same variance will be added to the input.
            If ``no`` is a tensor, it must have a shape that can be broadcast to
            the shape of ``x``. This allows, e.g., adding noise of different
            variance to each example in a batch. If ``no`` has a lower rank than
            ``x``, then ``no`` will be broadcast to the shape of ``x`` by adding
            dummy dimensions after the last axis.

    Output
    -------
        y : Tensor with same shape as ``x``, tf.complex
            Channel output
    """

    def __init__(self, dtype=tf.complex64, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self._real_dtype = tf.dtypes.as_dtype(self._dtype).real_dtype

    def call(self, inputs):

        x, no = inputs

        # Create tensors of real-valued Gaussian noise for each complex dim.
        noise = complex_normal(tf.shape(x), dtype=x.dtype)

        # Add extra dimensions for broadcasting
        no = expand_to_rank(no, tf.rank(x), axis=-1)

        # Apply variance scaling
        no = tf.cast(no, self._real_dtype)
        noise *= tf.cast(tf.sqrt(no), noise.dtype)

        # Add noise to input
        y = x + noise

        return y


2024-06-22 19:19:39.298224: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 19:19:39.354634: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 19:19:39.354663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 19:19:39.356185: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-22 19:19:39.365760: I tensorflow/core/platform/cpu_feature_guar

## example(tensorflow version)

In [5]:
# 创建 AWGN 实例
awgn_channel = AWGN()

# 定义输入
x_real = tf.random.normal([10, 2, 2, 14, 64])
x_imag = tf.random.normal([10, 2, 2, 14, 64])
x = tf.complex(x_real, x_imag)

no = tf.constant(0.01, dtype=tf.float32)  # 噪声方差

# 调用 AWGN 模块
y = awgn_channel((x, no))

# 打印输出
print(y)

tf.Tensor(
[[[[[-3.13242078e-02+3.54727060e-01j -4.24511284e-01+5.58363676e-01j
     -8.67400229e-01+5.40001392e-02j ...
      6.00697100e-01-9.59461272e-01j -1.20893133e+00+1.46220997e-01j
      4.12310004e-01-4.09173399e-01j]
    [-4.36643571e-01+1.15902352e+00j  1.79504395e+00+8.20372999e-01j
      8.45297635e-01+1.53562129e+00j ...
      4.87148464e-01+2.03382802e+00j -1.24476480e+00+1.19256818e+00j
     -3.32072496e-01-3.49351645e-01j]
    [ 6.72663033e-01+4.79259729e-01j -1.18728328e+00+5.00422120e-01j
      1.18328857e+00+5.98577857e-01j ...
     -1.31950593e+00-8.17314535e-03j  1.60577416e+00-1.09776664e+00j
      9.58612144e-01+8.42854857e-01j]
    ...
    [-5.29443324e-02+6.05568290e-04j -4.22519594e-01-1.22115982e+00j
     -6.21506035e-01-2.90435344e-01j ...
     -1.19006857e-01-1.72583270e+00j -1.43477368e+00-5.71747780e-01j
      9.23876107e-01-9.54831362e-01j]
    [ 1.30123273e-01+1.77215528e+00j  2.16297321e-02-4.65445757e-01j
     -6.71012819e-01-9.93700176e-02j ...
   

2024-06-22 19:19:42.318443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:31:00.0, compute capability: 8.9
